### Cab-Driver Agent

In [1]:
# Importing libraries
import numpy as np
import pandas as pd
import random
import math
from collections import deque
import collections
import pickle
import os
import time


# for building DQN model 
from tensorflow.keras import layers
from tensorflow.keras import Sequential
from tensorflow.keras.layers import Dense, Activation, Flatten
from tensorflow.keras.optimizers import Adam

# for plotting graphs
import matplotlib.pyplot as plt

# Import the environment
from Env import CabDriver

#### Defining Time Matrix

In [2]:
# Loading the time matrix provided
Time_matrix = np.load("TM.npy")

In [3]:
Time_matrix.shape

(5, 5, 24, 7)

In [4]:
Time_matrix[1][2][11][3]

10.0

In [5]:
Time_matrix.max()

11.0

Maximum time to travel between two locations is 11 hours (next state can change by max of 1 day)

#### Tracking the state-action pairs for checking convergence


In [6]:
#Defining a function to save the Q-dictionary as a pickle file
def save_obj(obj, name ):
    with open(name + '.pkl', 'wb') as f:
        pickle.dump(obj, f, pickle.HIGHEST_PROTOCOL)

### TODO  : Picking the architecture

#### Architecture 1: input - state, output - q-value for each possible (state, action)

Pros:
- Model has to be run once for each state  

Cons:
    

#### Architecture 2: input - (state, action), output - q-value for the given (state, action)

#### Conclusion

### Agent Class

If you are using this framework, you need to fill the following to complete the following code block:
1. State and Action Size
2. Hyperparameters
3. Create a neural-network model in function 'build_model()'
4. Define epsilon-greedy strategy in function 'get_action()'
5. Complete the function 'append_sample()'. This function appends the recent experience tuple <state, action, reward, new-state> to the memory
6. Complete the 'train_model()' function with following logic:
   - If the memory size is greater than mini-batch size, you randomly sample experiences from memory as per the mini-batch size and do the following:
      - Initialise your input and output batch for training the model
      - Calculate the target Q value for each sample: reward + gamma*max(Q(s'a,))
      - Get Q(s', a) values from the last trained model
      - Update the input batch as your encoded state and output batch as your Q-values
      - Then fit your DQN model using the updated input and output batch.

In [7]:
class DQNAgent:
    def __init__(self, state_size, action_size):
        # Define size of state and action
        self.state_size = state_size
        self.action_size = action_size

        # Write here: Specify you hyper parameters for the DQN 
        self.discount_factor = 0.99
        self.learning_rate = 0.001       
        self.epsilon_max = 1.0
        self.epsilon_decay = 0.999
        self.epsilon_min = 0.00000001
        self.epsilon = 1
    
        self.explore_count = 0
        self.exploit_count = 0
        
        self.batch_size = 32        
        # create replay memory using deque
        self.memory = deque(maxlen=2000)

        # Initialize the value of the states tracked
        self.states_tracked = []
        
        # Track the state [1,0,0] and action (1,3) at index 7 in the action space.
        self.track_state = np.array(env.state_encod_arch1([1,0,0])).reshape(1, 36)
        
        # create main model and target model
        self.model = self.build_model()
        
        
        
        

    # approximate Q function using Neural Network
    def build_model(self):
        model = Sequential()
        
        # input layer
        model.add(Dense(32, input_dim=self.state_size, activation='relu', kernel_initializer='he_uniform'))
        
        model.add(Dense(32, activation='relu', kernel_initializer='he_uniform'))
        model.add(Dense(32, activation='relu', kernel_initializer='he_uniform')) 
        
        # the output layer: output is of size num_actions
        model.add(Dense(self.action_size, activation='relu', kernel_initializer='he_uniform')) 
        
        
        
        model.compile(loss='mse',optimizer=Adam(lr=self.learning_rate))
        model.summary
        return model

     

    def get_action(self, state, episode_count):
    # Write your code here:
    # get action from model using epsilon-greedy policy
    # Decay in ε after we generate each sample from the environment       
        
        
#         exp_value = 1/10**(len(str(episode_count))-1)
#         self.epsilon = self.epsilon_min + (self.epsilon_max - self.epsilon_min) * np.exp(-exp_value*episode_count)
    
        
        
        if np.random.rand() <= self.epsilon:
            # exploration: choose a random action from all possible actions
            ##print(f"{episode_count}: EXPLORE")
            self.explore_count += 1
            return random.randrange(self.action_size)

        else:
            # exploitation: choose the action that returns the maximum q-value
            ##print(f"{episode_count}: EXPLOIT")
            
            
#             # the first index corresponds to the batch size, so
#             # reshape state to (1, state_size) so that the first index corresponds to the batch size 
            state = np.array(env.state_encod_arch1(state)).reshape(1, 36)
            #state = np.reshape(state,[1,self.state_size])
            q_value = self.model.predict(state) 
            self.exploit_count += 1
            return np.argmax(q_value)
         
 
   
    def get_summary_details(self):
        return self.explore_count, self.exploit_count
        

        
    def reset_episode_counts(self):
        self.explore_count = 0
        self.exploit_count = 0

    def append_sample(self, state, action_idx, reward, next_state, done):
    # Write your code here:
    # save sample <s,a,r,s',done> to the replay memory after every action
        self.memory.append((state, action_idx, reward, next_state, done))
        
        
#         # update ε after each sample
#         if self.epsilon > self.epsilon_min:
#             self.epsilon *= self.epsilon_decay    
    
    

    # Pick samples randomly from replay memory (with batch_size) and train the network
    def train_model(self):
        """
        train the neural network on a minibatch. Input to the network is the states,
        output is the target q-value corresponding to each action.
        """

        if len(self.memory) > self.batch_size:
            
            # sample minibatch from memory
            minibatch = random.sample(self.memory, self.batch_size)

            # initialise two matrices - update_input and update_output
            # Initialise the Q(s,a) with zero
            update_input = np.zeros((self.batch_size, self.state_size))
            # Initialise the Q(s',a)
            update_output = np.zeros((self.batch_size, self.state_size))
            actions, rewards, done = [], [], []

            # populate update_input and update_output and the lists rewards, actions, done
            for i in range(self.batch_size):
                state, action_idx, reward, next_state, is_done = minibatch[i]
                #print(f'4DEBUG:minibatch[{i}] = {minibatch[i]}')
                # Add state s to the Q(s,a), Q(s',a) from memory
                #print(f'4DEBUG: state:{state}, encoded state: {env.state_encod_arch1(state)}')
                update_input[i] = env.state_encod_arch1(state)     
                #update_input[i] = state
                # Add action from memory
                actions.append(action_idx)
                # Add reward from the memory
                rewards.append(reward)
                # Add next state s' to Q(s',a) from the memory
                update_output[i] = env.state_encod_arch1(next_state)
                #update_output[i] = next_state
                done.append(is_done)

            # Find the Q(s,a) and Q(s',a) using state as input to the neural network 
        
            # Predict the target q-values from state s
            target = self.model.predict(update_input)

            # Get the target for the Q-network
            target_qval = self.model.predict(update_output)
            
            #print(f'4DEBUG: target:{target}')
            #print(f'4DEBUG: target_qval:{target_qval}')

            # Update the target values - set the target as (r + maxQ(s',a))
            for i in range(self.batch_size):
                # Q Learning: get maximum Q value at s' from target model
                if done[i]:
                    target[i][actions[i]] = rewards[i]
                else: # non-terminal state
                    target[i][actions[i]] = rewards[i] + self.discount_factor * np.max(target_qval[i])

            # Train the model
            self.model.fit(update_input, target, batch_size=self.batch_size, epochs=1, verbose=0)
     

    def save_tracking_states(self):
        # Use the model to predict the q_value for the state tracked
        q_value = self.model.predict(self.track_state)
        #print(f'4DEBUG:save_tracking_states - {q_value}')
        # Update the q-value for the state tracked
        self.states_tracked.append(q_value[0][7])
        
    def save_model_weights(self, name):
        self.model.save_weights(name)

    def save(self, name):
        self.model.save(name)

In [8]:
Episodes = 10000

### DQN block

In [9]:

env = CabDriver()

m = 5
t = 24
d = 7
# get size of state and action from environment
state_size = m+t+d
action_size = len(env.action_space)

print(f'State size: {state_size}, Action size: {action_size}')

# agent needs to be initialised outside the loop since the DQN
# network will be initialised along with the agent
agent = DQNAgent(action_size=action_size, state_size=state_size) 

# to store rewards in each episode
rewards_per_episode, episodes = [], []

# make dir to store model weights
if not os.path.exists("saved_model_weights"):
    os.mkdir("saved_model_weights")
 

summary_df = pd.DataFrame(columns=['EPISODE','REWARD','MEMORY_LENGTH','EPSILON','EXPLORE_CNT', 'EXPLOIT_CNT', 'EPISODE_TIME'])

summary_threshold = 1000
def update_summary_details(episode,reward, memory_len, epsilon, explore_count, exploit_count,episode_time):
    #print(f'Updating summary details:{match_results}')
    return summary_df.append({'EPISODE' : episode,
                       'REWARD' : reward,
                       'MEMORY_LENGTH' : memory_len,
                       'EPSILON' : epsilon,
                       'EXPLORE_CNT' : explore_count,
                       'EXPLOIT_CNT' : exploit_count,
                       'EPISODE_TIME': episode_time
                      }, ignore_index=True) 

State size: 36, Action size: 21





for episode in range(Episodes):

    # Write code here
    # Call the environment
    # Call all the initialised variables of the environment
    

    #Call the DQN agent
    
    
    while !terminal_state:
        
        # Write your code here
        # 1. Pick epsilon-greedy action from possible actions for the current state
        # 2. Evaluate your reward and next state
        # 3. Append the experience to the memory
        # 4. Train the model by calling function agent.train_model
        # 5. Keep a track of rewards, Q-values, loss
        

In [ ]:
start_time = time.time()

#### simulation starts ####
for episode in range(Episodes):
    episode_start_time = time.time()
    done = False
    score = 0

    # reset at the start of each episode
    state = env.reset()

    while not done:

        # 1. Pick epsilon-greedy action from possible actions for the current state
        action_idx = agent.get_action(state,episode)
        action = env.action_space[action_idx]
        # 2. Evaluate your reward and next state
        next_state, reward, done = env.step(state, action, Time_matrix)
            
        #next_state, reward, done, info = env.step(action)

        # 3. Append the experience to the memory
        # save the sample <s, a, r, s', done> to the replay memory
        agent.append_sample(state, action_idx, reward, next_state, done)

        # 4. Train the model by calling function agent.train_model
        agent.train_model()

        # add reward to the total score of this episode
        score += reward
        state = next_state



    # epsilon decay
    if agent.epsilon > agent.epsilon_min:
        agent.epsilon *= agent.epsilon_decay
#         agent.epsilon = self.epsilon_min + (self.epsilon_max - self.epsilon_min) * np.exp(agent.epsilon_decay*episode)

    # 5. Keep a track of rewards, Q-values, loss
    # store total reward obtained in this episode
    rewards_per_episode.append(score)
    episodes.append(episode)
    
    # Save summary after every episode
    explore_count, exploit_count = agent.get_summary_details()
    episode_time = time.time() - episode_start_time
    summary_df = update_summary_details(episode,score, len(agent.memory),round(agent.epsilon,5),explore_count, exploit_count,round(episode_time,3))
    agent.reset_episode_counts()
        
    # every episode:
    print(f"episode {episode}, reward {score}, memory_length {len(agent.memory)}, epsilon {round(agent.epsilon,5)}, explore:{explore_count}, exploit:{exploit_count},time: {round(episode_time,3)}")
        
    # Save the Q_value of the state, action pair we are tracking
    if ((episode + 1) % 5 == 0):
        agent.save_tracking_states()
        
        
    # every few episodes:
    if episode % 100 == 0:
        # store q-values of some prespecified state-action pairs
        # q_dict = agent.store_q_values()
        
        
        curr_elapsed_time = time.time() - start_time
        print(f'Episodes:{episode} - Elapsed_time:{round(curr_elapsed_time,3)} ')  

        # save model weights
        agent.save_model_weights(name="model_weights.h5")

 
# save model weights
agent.save_model_weights(name="model_weights.h5")
    
elapsed_time = time.time() - start_time
print(f'Total elapsed_time:{elapsed_time}')       
    
        
#### simulation complete ####

episode 0, reward 146.0, memory_length 121, epsilon 0.999, explore:121, exploit:0,time: 7.662
Episodes:0 - Elapsed_time:7.665 
episode 1, reward -135.0, memory_length 244, epsilon 0.998, explore:123, exploit:0,time: 8.803
episode 2, reward 63.0, memory_length 363, epsilon 0.997, explore:119, exploit:0,time: 8.408
episode 3, reward -280.0, memory_length 482, epsilon 0.99601, explore:119, exploit:0,time: 8.547
episode 4, reward 35.0, memory_length 593, epsilon 0.99501, explore:111, exploit:0,time: 8.004
episode 5, reward 189.0, memory_length 720, epsilon 0.99401, explore:126, exploit:1,time: 9.106
episode 6, reward -257.0, memory_length 840, epsilon 0.99302, explore:120, exploit:0,time: 8.695
episode 7, reward 117.0, memory_length 947, epsilon 0.99203, explore:107, exploit:0,time: 7.586
episode 8, reward 362.0, memory_length 1058, epsilon 0.99104, explore:111, exploit:0,time: 7.916
episode 9, reward -398.0, memory_length 1165, epsilon 0.99004, explore:105, exploit:2,time: 7.788
episode 1

episode 84, reward -212.0, memory_length 2000, epsilon 0.91847, explore:115, exploit:7,time: 8.891
episode 85, reward -295.0, memory_length 2000, epsilon 0.91755, explore:107, exploit:6,time: 8.105
episode 86, reward -108.0, memory_length 2000, epsilon 0.91664, explore:117, exploit:6,time: 8.933
episode 87, reward 250.0, memory_length 2000, epsilon 0.91572, explore:117, exploit:11,time: 9.502
episode 88, reward -32.0, memory_length 2000, epsilon 0.9148, explore:113, exploit:13,time: 9.315
episode 89, reward -63.0, memory_length 2000, epsilon 0.91389, explore:111, exploit:5,time: 8.534
episode 90, reward -177.0, memory_length 2000, epsilon 0.91298, explore:110, exploit:9,time: 8.899
episode 91, reward 20.0, memory_length 2000, epsilon 0.91206, explore:103, exploit:8,time: 8.025
episode 92, reward 211.0, memory_length 2000, epsilon 0.91115, explore:105, exploit:11,time: 8.614
episode 93, reward -5.0, memory_length 2000, epsilon 0.91024, explore:100, exploit:11,time: 8.151
episode 94, rew

episode 167, reward 141.0, memory_length 2000, epsilon 0.84528, explore:114, exploit:21,time: 9.996
episode 168, reward 90.0, memory_length 2000, epsilon 0.84444, explore:101, exploit:20,time: 9.038
episode 169, reward -243.0, memory_length 2000, epsilon 0.84359, explore:93, exploit:22,time: 8.465
episode 170, reward -407.0, memory_length 2000, epsilon 0.84275, explore:99, exploit:19,time: 8.74
episode 171, reward -207.0, memory_length 2000, epsilon 0.84191, explore:91, exploit:15,time: 7.697
episode 172, reward -122.0, memory_length 2000, epsilon 0.84106, explore:84, exploit:15,time: 7.311
episode 173, reward -202.0, memory_length 2000, epsilon 0.84022, explore:97, exploit:19,time: 8.759
episode 174, reward 20.0, memory_length 2000, epsilon 0.83938, explore:101, exploit:17,time: 8.709
episode 175, reward -371.0, memory_length 2000, epsilon 0.83854, explore:103, exploit:13,time: 8.509
episode 176, reward -154.0, memory_length 2000, epsilon 0.83771, explore:105, exploit:13,time: 8.628
e

episode 250, reward 0.0, memory_length 2000, epsilon 0.77792, explore:92, exploit:25,time: 8.856
episode 251, reward -215.0, memory_length 2000, epsilon 0.77715, explore:93, exploit:20,time: 8.287
episode 252, reward -161.0, memory_length 2000, epsilon 0.77637, explore:92, exploit:19,time: 8.273
episode 253, reward 35.0, memory_length 2000, epsilon 0.77559, explore:99, exploit:16,time: 8.511
episode 254, reward -72.0, memory_length 2000, epsilon 0.77482, explore:106, exploit:17,time: 8.923
episode 255, reward 121.0, memory_length 2000, epsilon 0.77404, explore:95, exploit:22,time: 8.731
episode 256, reward 81.0, memory_length 2000, epsilon 0.77327, explore:70, exploit:31,time: 7.754
episode 257, reward 192.0, memory_length 2000, epsilon 0.7725, explore:84, exploit:28,time: 8.568
episode 258, reward 52.0, memory_length 2000, epsilon 0.77172, explore:83, exploit:29,time: 8.654
episode 259, reward -9.0, memory_length 2000, epsilon 0.77095, explore:77, exploit:25,time: 7.706
episode 260, r

episode 333, reward -99.0, memory_length 2000, epsilon 0.71593, explore:91, exploit:28,time: 8.888
episode 334, reward 102.0, memory_length 2000, epsilon 0.71522, explore:74, exploit:36,time: 8.627
episode 335, reward 86.0, memory_length 2000, epsilon 0.7145, explore:72, exploit:29,time: 7.696
episode 336, reward -122.0, memory_length 2000, epsilon 0.71379, explore:72, exploit:29,time: 7.789
episode 337, reward 297.0, memory_length 2000, epsilon 0.71307, explore:87, exploit:20,time: 8.021
episode 338, reward 378.0, memory_length 2000, epsilon 0.71236, explore:82, exploit:35,time: 8.998
episode 339, reward 180.0, memory_length 2000, epsilon 0.71165, explore:85, exploit:38,time: 9.562
episode 340, reward 153.0, memory_length 2000, epsilon 0.71094, explore:71, exploit:31,time: 8.047
episode 341, reward 562.0, memory_length 2000, epsilon 0.71023, explore:70, exploit:31,time: 7.872
episode 342, reward 147.0, memory_length 2000, epsilon 0.70952, explore:70, exploit:30,time: 7.876
episode 343

...

episode 5118, reward 2345.0, memory_length 2000, epsilon 0.00597, explore:1, exploit:102,time: 9.837
episode 5119, reward 2494.0, memory_length 2000, epsilon 0.00596, explore:0, exploit:110,time: 10.498
episode 5120, reward 2265.0, memory_length 2000, epsilon 0.00595, explore:1, exploit:109,time: 10.435
episode 5121, reward 2328.0, memory_length 2000, epsilon 0.00595, explore:1, exploit:106,time: 10.156
episode 5122, reward 2053.0, memory_length 2000, epsilon 0.00594, explore:0, exploit:101,time: 9.665
episode 5123, reward 1647.0, memory_length 2000, epsilon 0.00594, explore:0, exploit:101,time: 9.478
episode 5124, reward 1171.0, memory_length 2000, epsilon 0.00593, explore:2, exploit:93,time: 9.036
episode 5125, reward 2219.0, memory_length 2000, epsilon 0.00593, explore:0, exploit:107,time: 10.179
episode 5126, reward 1995.0, memory_length 2000, epsilon 0.00592, explore:0, exploit:106,time: 10.141
episode 5127, reward 2007.0, memory_length 2000, epsilon 0.00591, explore:0, exploit:10

episode 5199, reward 2778.0, memory_length 2000, epsilon 0.0055, explore:0, exploit:112,time: 10.74
episode 5200, reward 2778.0, memory_length 2000, epsilon 0.0055, explore:0, exploit:112,time: 10.766
Episodes:5200 - Elapsed_time:56180.878 
episode 5201, reward 2778.0, memory_length 2000, epsilon 0.00549, explore:0, exploit:112,time: 10.734
episode 5202, reward 2751.0, memory_length 2000, epsilon 0.00549, explore:0, exploit:112,time: 10.529
episode 5203, reward 2796.0, memory_length 2000, epsilon 0.00548, explore:1, exploit:113,time: 10.896
episode 5204, reward 2670.0, memory_length 2000, epsilon 0.00547, explore:1, exploit:111,time: 10.647
episode 5205, reward 2751.0, memory_length 2000, epsilon 0.00547, explore:0, exploit:112,time: 10.717
episode 5206, reward 2778.0, memory_length 2000, epsilon 0.00546, explore:0, exploit:112,time: 10.746
episode 5207, reward 2747.0, memory_length 2000, epsilon 0.00546, explore:1, exploit:120,time: 11.818
episode 5208, reward 2778.0, memory_length 20

episode 5280, reward 2274.0, memory_length 2000, epsilon 0.00507, explore:1, exploit:115,time: 10.88
episode 5281, reward 2208.0, memory_length 2000, epsilon 0.00507, explore:0, exploit:116,time: 11.137
episode 5282, reward 2191.0, memory_length 2000, epsilon 0.00506, explore:2, exploit:134,time: 12.893
episode 5283, reward 2490.0, memory_length 2000, epsilon 0.00506, explore:2, exploit:110,time: 10.709
episode 5284, reward 2475.0, memory_length 2000, epsilon 0.00505, explore:1, exploit:105,time: 10.122
episode 5285, reward 2625.0, memory_length 2000, epsilon 0.00505, explore:0, exploit:115,time: 10.991
episode 5286, reward 2348.0, memory_length 2000, epsilon 0.00504, explore:0, exploit:109,time: 10.298
episode 5287, reward 2502.0, memory_length 2000, epsilon 0.00504, explore:1, exploit:133,time: 12.784
episode 5288, reward 2674.0, memory_length 2000, epsilon 0.00503, explore:1, exploit:102,time: 9.862
episode 5289, reward 2674.0, memory_length 2000, epsilon 0.00503, explore:0, exploit

episode 5361, reward 1957.0, memory_length 2000, epsilon 0.00468, explore:0, exploit:110,time: 10.547
episode 5362, reward 2298.0, memory_length 2000, epsilon 0.00467, explore:1, exploit:107,time: 10.131
episode 5363, reward 2180.0, memory_length 2000, epsilon 0.00467, explore:2, exploit:112,time: 10.854
episode 5364, reward 2360.0, memory_length 2000, epsilon 0.00466, explore:0, exploit:109,time: 10.404
episode 5365, reward 1964.0, memory_length 2000, epsilon 0.00466, explore:2, exploit:138,time: 13.418
episode 5366, reward 2092.0, memory_length 2000, epsilon 0.00466, explore:2, exploit:103,time: 9.848
episode 5367, reward 1957.0, memory_length 2000, epsilon 0.00465, explore:0, exploit:99,time: 9.439
episode 5368, reward 2254.0, memory_length 2000, epsilon 0.00465, explore:0, exploit:113,time: 10.614
episode 5369, reward 1899.0, memory_length 2000, epsilon 0.00464, explore:1, exploit:113,time: 10.863
episode 5370, reward 2277.0, memory_length 2000, epsilon 0.00464, explore:1, exploit:

episode 5442, reward 2056.0, memory_length 2000, epsilon 0.00431, explore:0, exploit:102,time: 9.744
episode 5443, reward 2192.0, memory_length 2000, epsilon 0.00431, explore:0, exploit:107,time: 10.159
episode 5444, reward 2336.0, memory_length 2000, epsilon 0.00431, explore:1, exploit:98,time: 9.44
episode 5445, reward 2228.0, memory_length 2000, epsilon 0.0043, explore:1, exploit:93,time: 8.796
episode 5446, reward 2369.0, memory_length 2000, epsilon 0.0043, explore:0, exploit:106,time: 10.09
episode 5447, reward 2345.0, memory_length 2000, epsilon 0.00429, explore:1, exploit:102,time: 9.834
episode 5448, reward 2344.0, memory_length 2000, epsilon 0.00429, explore:1, exploit:104,time: 10.02
episode 5449, reward 2119.0, memory_length 2000, epsilon 0.00428, explore:1, exploit:108,time: 10.364
episode 5450, reward 2308.0, memory_length 2000, epsilon 0.00428, explore:0, exploit:107,time: 10.087
episode 5451, reward 2243.0, memory_length 2000, epsilon 0.00428, explore:2, exploit:101,time

In [ ]:

plt.figure(0, figsize=(16,8))
plt.plot(summary_df[['EPISODE','REWARD']])
plt.title('REWARDS PER EPISODE')
plt.xlabel("Episode")
plt.ylabel("Reward")

plt.show()

# save plots in saved_plots/ directory
plt.savefig('rewards_episodes.png')

In [ ]:
display(summary_df)

In [ ]:
# save stuff as pickle
def save_pickle(obj, name):
    with open(name + '.pkl', 'wb') as f:
        pickle.dump(obj, f, pickle.HIGHEST_PROTOCOL)

# make directory
if not os.path.exists("saved_pickle_files"):
    os.mkdir("saved_pickle_files")

# save rewards_per_episode
save_pickle(rewards_per_episode, "saved_pickle_files/rewards_per_episode")
save_pickle(summary_df, "saved_pickle_files/summary_per_episode")



### Tracking Convergence

#### Tracked state

In [ ]:
agent.states_tracked

In [ ]:
state_tracked_sample = [agent.states_tracked[i] for i in range(len(agent.states_tracked)) if agent.states_tracked[i] < 1000]

#### Q-value convergence plot for the tracked state

In [ ]:
plt.figure(0, figsize=(16,7))
plt.title('Q_value for state [1,0,0]  action (1,3)')
xaxis = np.asarray(range(0, len(agent.states_tracked)))
plt.semilogy(xaxis,np.asarray(agent.states_tracked))
plt.show()

#### Plot Episode vs Reward

In [ ]:

# plot results
with open('saved_pickle_files/summary_per_episode.pkl', 'rb') as f:
    summary_data = pickle.load(f)

    
plt.figure(0, figsize=(16,8))
plt.plot(summary_data[['EPISODE','REWARD']])
plt.title('REWARDS PER EPISODE')
plt.xlabel("Episode")
plt.ylabel("Reward")

plt.show()

# save plots in saved_plots/ directory
plt.savefig('rewards_episodes.png')
  

In [ ]:

# plot results
with open('saved_pickle_files/rewards_per_episode.pkl', 'rb') as f:
    rewards_per_episode = pickle.load(f)

plt.plot(list(range(len(rewards_per_episode))), rewards_per_episode)
plt.xlabel("episode number")
plt.ylabel("reward per episode")

plt.show()

# save plots in saved_plots/ directory
plt.savefig('rewards.png')


print("Average reward of last 100 episodes is {0}".format(np.mean(rewards_per_episode[-100:]))) 









#### Plot exploration vs exploitation

In [ ]:

plt.figure(0, figsize=(16,8))
plt.plot(summary_data[['EXPLORE_CNT','EXPLOIT_CNT']])
plt.title('EXPLORATION vs EXPLOITATION')
plt.xlabel("Exploration")
plt.ylabel("Exploitation")

plt.show()

# save plots in saved_plots/ directory
plt.savefig('exploration_exploitation.png')

### Epsilon-decay sample function

<div class="alert alert-block alert-info">
Try building a similar epsilon-decay function for your model.
</div>

In [ ]:
import matplotlib.pyplot as plt

max_epsilon = 1.0
min_epsilon = 0.0000001
episode_count = 25000
exp_value = 1/10**(len(str(episode_count))-1)
print(f'exp:{exp_value}')
time = np.arange(0,episode_count)
epsilon = []
for i in range(0,episode_count):
    epsilon.append(min_epsilon + (max_epsilon - min_epsilon) * np.exp(-exp_value*i))
    

plt.plot(time, epsilon)
plt.show()

In [ ]:
# From starter code
total_episodes = 10000
time = np.arange(0,total_episodes)
epsilon = []
for i in range(0,total_episodes):
    epsilon.append(0 + (1 - 0) * np.exp(-0.0009*i))
    
plt.plot(time, epsilon)
plt.show()

In [ ]:
max_epsilon = 1.0
min_epsilon = 0.001
episode_count = 50
exp_value = 1/10**(len(str(episode_count))-1)
time = np.arange(0,episode_count)
epsilon = []
for i in range(0,episode_count):
    epsilon.append(min_epsilon + (max_epsilon - min_epsilon) * np.exp(-exp_value*i))
    

plt.plot(time, epsilon)
plt.show()